In [1]:
from dotenv import load_dotenv
import os

import pandas as pd
import requests


load_dotenv()

True

# Константы

In [2]:
TOKEN_MOSCOW = os.getenv('TOKEN_MOSCOW')

In [3]:
URL_BASE = 'apidata.mos.ru'

In [4]:
params = {
    'api_key': TOKEN_MOSCOW
}

## Получить текущую версию API

In [5]:
url_version = f'https://{URL_BASE}/version'

In [6]:
response = requests.get(url_version, params=params)

In [7]:
print('Версия API: ', response.json().get('Version'))

Версия API:  1


## Получить список наборов данных

In [8]:
url_datasets = f'https://{URL_BASE}/v1/datasets'

In [9]:
params_datasets = params.copy()
params_datasets['$inlinecount'] = 'allpages'
params_datasets['$top'] = 20


In [10]:
response = requests.get(url_datasets, params=params_datasets)

In [11]:
df_datasets = pd.DataFrame(response.json().get('Items'))

In [12]:
df_datasets.tail()

,CategoryId,PublishDate,Keywords,IdentificationNumber,ContainsGeodata,IsSeasonal,IsNew,VersionNumber,Caption,ContainsAccEnvData,IsForeign,IsArchive,SefUrl,Id,ReleaseNumber,DepartmentId,Season,FullDescription,LastUpdateDate
15,104,2013-10-07,"бесплатное детское питание, грудничок, детский...",7707089084-Dairyspecialties,True,False,False,2,Молочные кухни,True,False,False,None,512,390,1002,0,"<p style=""margin-left:0cm; margin-right:0cm; t...",2025-01-06
16,104,2013-10-07,"акушеры, беременность, здоровье, здравоохранен...",7707089084-MaternityHomes,True,False,False,2,Родильные дома,True,False,False,None,514,361,1002,0,"<p style=""margin-left:0cm; margin-right:0cm; t...",2024-12-27
17,104,2013-10-07,"донор, донорство, здоровье, здравоохранение, к...",7707089084-donation,True,False,False,2,Станции переливания крови,True,False,False,None,515,320,1002,0,"<p style=""margin-left:0cm; margin-right:0cm; t...",2024-12-27
18,104,2013-10-31,"бригада скорой помощи, вызвать врача, вызов ск...",7707089084-medicalemergency,True,False,False,2,Скорая и неотложная медицинская помощь,True,False,False,None,516,169,1002,0,"<p style=""margin-left:0cm; margin-right:0cm; t...",2025-01-03
19,104,2013-10-07,"больница, больница клиническая, больницы взрос...",7707089084-clinics,True,False,False,2,Больницы взрослые,True,False,False,None,517,522,1002,0,"<p><span style=""font-family:tahoma,geneva,sans...",2024-12-27


## Получить содержимое наборов данных

In [13]:
ID_DATASET = 658

url_datasets_rows = f'https://{URL_BASE}/v1/datasets/{ID_DATASET}/rows'

In [14]:
params_datasets_rows = params.copy()
params_datasets_rows['$orderby'] = 'global_id'
params_datasets_rows['$top'] = 1000

In [15]:
response = requests.get(url_datasets_rows, params=params_datasets_rows)

Обработка выгруженного диапазона в data_tickets

Выбор нужного содержимого

In [23]:
response.json()[0].get('Cells')

{'ID': 200,
 'NameOfTariff': '«Единый» не более 2 поездок',
 'TicketZone': [{'is_deleted': 0, 'TicketZone': 'Зона А', 'global_id': 1553},
  {'is_deleted': 0, 'TicketZone': 'Зона Б', 'global_id': 1877},
  {'is_deleted': 0, 'TicketZone': 'между зонами А и Б', 'global_id': 1878},
  {'is_deleted': 0, 'TicketZone': 'МЦК', 'global_id': 1879}],
 'global_id': 19368709,
 'NumberOfZone': '-',
 'TariffDistance': '-',
 'TypeOfTransport': 'Метрополитен, монорельс, МЦК, автобус, трамвай, электробус',
 'NameOfCarrier': [{'is_deleted': 0,
   'NameOfCarrier': 'ГУП «Московский метрополитен»',
   'global_id': 993},
  {'is_deleted': 0, 'NameOfCarrier': 'ГУП «Мосгортранс»', 'global_id': 994},
  {'is_deleted': 0,
   'NameOfCarrier': 'перевозчики наземного транспорта, работающие по государственным контрактам',
   'global_id': 1664},
  {'is_deleted': 0, 'NameOfCarrier': 'ОАО «РЖД»', 'global_id': 1846}],
 'TicketCost': 140,
 'TicketValidity': '5 дней, включая день продажи'}

Мы видим что имеется несколько ticket zone и перевозчиков

In [27]:
max_tickets = 0
max_carrier = 0

for elem in response.json():
    cell = elem.get('Cells')
    count_tickets = len(cell.get('TicketZone'))
    count_carrier = len(cell.get('NameOfCarrier'))
    max_tickets = max(count_tickets, max_tickets)
    max_carrier = max(count_carrier, max_carrier)

In [29]:
max_tickets, max_carrier

(5, 6)

Итого необходимо будет создат 5 столбцов с зонами для билетов и 6 перевозчиков

In [45]:
data_tickets = []

In [46]:
for elem in response.json():
    cell = elem.get('Cells')

    ticket_zones = cell.get('TicketZone', [])
    name_zones = []
    for zone in ticket_zones:
        name_zones.append(zone.get('TicketZone'))

    carrier_data = cell.get('NameOfCarrier', [])
    name_carriers = []
    for carrier in carrier_data:
        name_carriers.append(carrier.get('NameOfCarrier'))

    data_tickets.append({
        'global_id': elem.get('global_id'),
        'id_cell' : cell.get('ID'),
        'name_zones': ', '.join(name_zones),
        'name_carriers': ', '.join(name_carriers),
        'name_of_tariff': cell.get('NameOfTariff'),
        'type_of_transport': cell.get('TypeOfTransport'),
        'ticket_cost': cell.get('TicketCost'),
        'ticket_validity': cell.get('TicketValidity'),
    })

In [51]:
data_tickets[:3]

[{'global_id': 19368709,
  'id_cell': 200,
  'name_zones': 'Зона А, Зона Б, между зонами А и Б, МЦК',
  'name_carriers': 'ГУП «Московский метрополитен», ГУП «Мосгортранс», перевозчики наземного транспорта, работающие по государственным контрактам, ОАО «РЖД»',
  'name_of_tariff': '«Единый» не более 2 поездок',
  'type_of_transport': 'Метрополитен, монорельс, МЦК, автобус, трамвай, электробус',
  'ticket_cost': 140,
  'ticket_validity': '5 дней, включая день продажи'},
 {'global_id': 19368716,
  'id_cell': 700,
  'name_zones': 'Зона А, Зона Б, между зонами А и Б',
  'name_carriers': 'ГУП «Московский метрополитен», ГУП «Мосгортранс», АО «Центральная пригородная пассажирская компания», перевозчики наземного транспорта, работающие по государственным контрактам, АО «Московско-Тверская пригородная пассажирская компания», ОАО «РЖД»',
  'name_of_tariff': '«Единый» не более 60 поездок',
  'type_of_transport': 'Метрополитен, монорельс, МЦК, автобус, трамвай, электробус, МЦД в зоне Центральная',
 

In [48]:
df_tickets = pd.DataFrame(data_tickets)

In [49]:
df_columns = [
    "global_id", "id строки", "Наименование посадочных зон", "Наименование перевозчиков", 
    'Наименование тарифа', 'Типа транспорта','Стоимость билетов', 'Начало активации'
]

df_tickets.columns = df_columns

In [50]:
df_tickets.sort_values(by='Стоимость билетов', ascending=False)[:10]

,global_id,id строки,Наименование посадочных зон,Наименование перевозчиков,Наименование тарифа,Типа транспорта,Стоимость билетов,Начало активации
20,1029846586,8100,"Зона А, Зона Б (в части ЗелАО), МЦК, МЦД","ГУП «Московский метрополитен», ГУП «Мосгортран...",«Единый» на 365 дней,"Метрополитен, монорельс, МЦК, автобус, трамвай...",25700,"365 дней с момент первого прохода, но не более..."
5,19368725,1300,"Зона А, Зона Б (в части ЗелАО), МЦК, МЦД","ГУП «Московский метрополитен», ГУП «Мосгортран...",«Единый» на 365 дней,"Метрополитен, монорельс, МЦК, автобус, трамвай...",20500,"365 дней с момент первого прохода, но не более..."
26,2388962130,9100,"Зона А, Зона Б (в части ЗелАО)","ГУП «Мосгортранс», перевозчики наземного транс...",«ТАТ» на 365 дней,"Автобус, электробус, трамвай",14750,"365 дней с момент первого прохода, но не более..."
19,1029846584,8000,"Зона А, Зона Б (в части ЗелАО), МЦК, МЦД","ГУП «Московский метрополитен», ГУП «Мосгортран...",«Единый» на 90 дней,"Метрополитен, монорельс, МЦК, автобус, трамвай...",8850,"90 дней с момент первого прохода, но не более ..."
4,19368723,1200,"Зона А, Зона Б (в части ЗелАО), МЦК, МЦД","ГУП «Московский метрополитен», ГУП «Мосгортран...",«Единый» на 90 дней,"Метрополитен, монорельс, МЦК, автобус, трамвай...",6950,"90 дней с момент первого прохода, но не более ..."
25,2388962108,9000,"Зона А, Зона Б (в части ЗелАО)","ГУП «Мосгортранс», перевозчики наземного транс...",«ТАТ» на 90 дней,"Автобус, электробус, трамвай",4800,"90 дней с момент первого прохода, но не более ..."
33,2608943181,9504,"МЦК, Зона А, Зона Б (в части ЗелАО), ММЦД","ГУП «Мосгортранс», АО «Центральная пригородная...",Единый льготный проездной билет для обучающихс...,"Метрополитен, монорельс, МЦК, автобус, трамвай...",3610,"90 дней с момент первого прохода, но не более ..."
18,1029846583,7800,"Зона А, Зона Б (в части ЗелАО), МЦК, МЦД","ГУП «Московский метрополитен», ГУП «Мосгортран...",«Единый» на 30 дней,"Метрополитен, монорельс, МЦК, автобус, трамвай...",3450,"30 дней с момент первого прохода, но не более ..."
1,19368716,700,"Зона А, Зона Б, между зонами А и Б","ГУП «Московский метрополитен», ГУП «Мосгортран...",«Единый» не более 60 поездок,"Метрополитен, монорельс, МЦК, автобус, трамвай...",3420,"45 дней, включая день продажи"
3,19368722,1100,"Зона А, Зона Б (в части ЗелАО), МЦК, МЦД","ГУП «Московский метрополитен», ГУП «Мосгортран...",«Единый» на 30 дней,"Метрополитен, монорельс, МЦК, автобус, трамвай...",2870,"30 дней с момент первого прохода, но не более ..."
